In [75]:
%reset -f
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
from linearmodels.iv import IV2SLS
import functions as fun
import statsmodels.formula.api as sm
import seaborn as sns
from matplotlib import pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [76]:
df = pd.read_csv('dataset.csv')
Nobs=df['ID'].count()
df['Intercept']=np.ones((Nobs,1))
df.rename(columns={'Market share':'Market_share'}, inplace=True)
df2 = df[df['Market_share'] != 0]

df2.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
8,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
9,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
10,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
21,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
28,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
29,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
30,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
31,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
32,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
41,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [77]:
# Copy the dataframe
data = df2.copy().reset_index(drop=True)
data.head(20)

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept
0,1,2021,0.010373,Aiways,U5,400,284621.700,201,34,SUV,C,CN,257,1.0
1,1,2022,0.005976,Aiways,U5,400,313681.829,201,34,SUV,C,CN,183,1.0
2,1,2023,0.002860,Aiways,U5,400,264524.000,201,34,SUV,C,CN,177,1.0
3,2,2023,0.000048,Aiways,U6,405,360638.000,214,34,SUV,C,CN,3,1.0
4,3,2019,0.040630,Audi,e-tron,375,979704.475,402,17,SUV,F,DE,222,1.0
5,3,2020,0.034680,Audi,e-tron,375,890101.410,402,17,SUV,F,DE,491,1.0
6,3,2021,0.010494,Audi,e-tron,375,800035.193,402,17,SUV,F,DE,260,1.0
7,3,2022,0.017570,Audi,e-tron,375,789723.656,402,17,SUV,F,DE,538,1.0
8,3,2023,0.001099,Audi,e-tron,375,673037.728,402,17,SUV,F,DE,68,1.0
9,4,2021,0.003391,Audi,e-tron GT,472,1278896.110,522,17,Sedan,F,DE,84,1.0


In [78]:
#Scale for better intepretation
data['Price'] = data['Price']/10_000 #(Change in ms(%) for change in pris in 10.000)
data['HP'] = data['HP']/10           #(Change in ms(%) for change in HP in 10)
data['Range'] = data['Range']/10     #(Change in ms(%) for change in rækkevidde in 10)

In [79]:
#Creating dummy for china
data['China'] = (data['Country'] == 'CN').astype(int)

In [80]:
data = data.sort_values(['Year', 'ID']).reset_index(drop=True)

# OLS

In [81]:
OLS = sm.ols('np.log(Market_share) ~ Price + Range + HP + Chargetime + China + C(Year) + C(Model)', data).fit(cov_type='HC3')
OLS.summary()#.tables[1]

/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 121, but rank is 117
  warnings.warn('covariance of constraints does not have full '


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                             
================================================================================
Dep. Variable:     np.log(Market_share)   R-squared:                       0.783
Model:                              OLS   Adj. R-squared:                  0.666
Method:                   Least Squares   F-statistic:                     568.6
Date:                  Thu, 23 May 2024   Prob (F-statistic):          1.72e-224
Time:                          23:33:26   Log-Likelihood:                -499.83
No. Observations:                   334   AIC:                             1236.
Df Residuals:                       216   BIC:                             1685.
Df Model:                           117                                         
Covariance Type:                    HC3                                         
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                        -2.8836    138.576     -0.021      0.983    -274.487     268.720
C(Year)[T.2014]                   0.4805      0.843      0.570      0.569      -1.171       2.132
C(Year)[T.2015]                  -0.0019      0.997     -0.002      0.999      -1.956       1.952
C(Year)[T.2016]                   0.4884      0.828      0.590      0.555      -1.134       2.111
C(Year)[T.2017]                   0.3420      0.853      0.401      0.689      -1.330       2.014
C(Year)[T.2018]                  -0.3734      0.907     -0.412      0.681      -2.151       1.404
C(Year)[T.2019]                  -0.9812      0.812     -1.208      0.227      -2.573       0.611
C(Year)[T.2020]                  -1.9431      0.838     -2.319      0.020      -3.585      -0.301
C(Year)[T.2021]                  -2.9092      0.850     -3.422      0.001      -4.575      -1.243
C(Year)[T.2022]                  -3.0676      0.824     -3.721      0.000      -4.684      -1.452
C(Year)[T.2023]                  -3.4590      0.838     -4.126      0.000      -5.102      -1.816
C(Model)[T.2008]                  0.5707     30.435      0.019      0.985     -59.081      60.223
C(Model)[T.208]                   0.8510     26.984      0.032      0.975     -52.036      53.738
C(Model)[T.3 Crossback]          -1.5637     34.040     -0.046      0.963     -68.281      65.154
C(Model)[T.3-dørs]                1.3363     57.268      0.023      0.981    -110.907     113.579
C(Model)[T.4]                     1.5124     61.251      0.025      0.980    -118.537     121.562
C(Model)[T.5]                     0.7200     61.789      0.012      0.991    -120.383     121.823
C(Model)[T.500]                   0.3411     36.027      0.009      0.992     -70.271      70.954
C(Model)[T.Ariya]                 1.0444     22.966      0.045      0.964     -43.968      46.056
C(Model)[T.Atto 3]                0.7701     65.253      0.012      0.991    -127.124     128.664
C(Model)[T.Avenger]              -1.1827     81.611     -0.014      0.988    -161.138     158.772
C(Model)[T.Berlingo]             -0.6033     43.381     -0.014      0.989     -85.629      84.423
C(Model)[T.Born]                  1.0551     18.937      0.056      0.956     -36.061      38.172
C(Model)[T.C4]                    0.2260     27.328      0.008      0.993     -53.335      53.787
C(Model)[T.C40]                  -1.0764     12.079     -0.089      0.929     -24.751      22.598
C(Model)[T.Cabriolet]            -1.3787    773.430     -0.002      0.999   -1517.274    1514.516
C(Model)[T.Citigo]               -1.0740     27.786     -0.039      0.969     -55.534      53.386
C(Model)[T.Combo]                -2.3128     43.865     -0.053      0.958     -88.286      83.661
C(Model)[T.Corsa]      

# Willingness to pay

In [82]:
alpha = OLS.params[-5] # Price coefficient
beta = list(np.concatenate((OLS.params[:1],OLS.params[-4:])))
beta_alpha_ratio = [b / -alpha for b in beta[1:]] #Willingness to pay (excluding constant)

for i in range(len(beta[1:])): #[2:] to exclude constant and price
    print('W2P:', OLS.params.index[-4:][i], beta_alpha_ratio[i])

W2P: Range 3.8034804318535977
W2P: HP -1.0022926086145119
W2P: Chargetime -2.2925695420286725
W2P: China -53.83019242725728


NOTE: English delimiter\
38,034 DKK for every 10 increase in Range\
-10,022 DKK for every 10 increase in HP\
-22,925 DKK for every 10 increase in Chargetime\
-538,301 DKK for Chinese cars

# Logit

In [83]:
# IMPORTANT: The data must be sorted by year and ID before running the function
#             Because the CCPs returned by the function are sorted by year and ID
logit_data = data.sort_values(['Year', 'ID']).reset_index(drop=True)
X = logit_data[['Intercept', 'Range', 'HP', 'Chargetime', 'China']]
p_j = logit_data['Price']
logit_data['CCP'] = fun.ccp(alpha, beta, data, X)

In [84]:
probability_ratio = fun.probability_ratio(logit_data, 2023)
probability_ratio

Model,U5,U6,e-tron,e-tron GT,Q4 e-tron,Q8 e-tron,I3,i4,i5,I7,...,ID.3,ID.4,ID.5,ID.7,ID.Buzz,up!,C40,EX30,XC40,Free
Model,,,,,,,,,,,,,,,,,,,,,
U5,1.0,1.216163,0.616206,1.17601,0.292087,0.352013,0.346933,0.230552,0.225576,2.429903,...,0.253282,0.165645,0.147724,0.088774,0.405584,1.298832,0.296879,0.179795,0.325815,3.868019
U6,0.822258,1.0,0.50668,0.966983,0.240171,0.289446,0.285268,0.189574,0.185482,1.998007,...,0.208263,0.136203,0.121467,0.072995,0.333495,1.067975,0.244111,0.147838,0.267904,3.18051
e-tron,1.622834,1.97363,1.0,1.908468,0.474008,0.571258,0.563014,0.374148,0.366072,3.943328,...,0.411035,0.268814,0.239731,0.144065,0.658196,2.107788,0.481784,0.291778,0.528744,6.277152
e-tron GT,0.850333,1.034144,0.52398,1.0,0.248371,0.299328,0.295008,0.196046,0.191815,2.066227,...,0.215374,0.140853,0.125614,0.075487,0.344882,1.10444,0.252446,0.152886,0.277052,3.289105
Q4 e-tron,3.423639,4.163704,2.109667,4.026233,1.0,1.205165,1.187773,0.789328,0.772291,8.319111,...,0.867147,0.567108,0.505753,0.30393,1.388574,4.446731,1.016405,0.615554,1.115474,13.242701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
up!,0.769923,0.936352,0.474431,0.905437,0.224884,0.271023,0.267111,0.177508,0.173676,1.870838,...,0.195008,0.127534,0.113736,0.068349,0.312269,1.0,0.228574,0.138428,0.250853,2.978076
C40,3.368381,4.096501,2.075617,3.961249,0.98386,1.185714,1.168602,0.776589,0.759826,8.18484,...,0.853151,0.557955,0.49759,0.299024,1.366162,4.37496,1.0,0.605619,1.09747,13.028963
EX30,5.561882,6.764156,3.427266,6.540827,1.624553,1.957855,1.929599,1.282306,1.254627,13.514834,...,1.408726,0.921297,0.821622,0.49375,2.255812,7.223949,1.651203,1.0,1.812147,21.513467


In [85]:
logit_data['Model_year'] = logit_data['Model'] + '_' + logit_data['Year'].astype(str)
marginal_effects = fun.marginal_effects(logit_data, OLS)
marginal_effects

,Intercept,C(Year)[T.2014],C(Year)[T.2015],C(Year)[T.2016],C(Year)[T.2017],C(Year)[T.2018],C(Year)[T.2019],C(Year)[T.2020],C(Year)[T.2021],C(Year)[T.2022],...,C(Model)[T.iX],C(Model)[T.iX1],C(Model)[T.iX3],C(Model)[T.up!],C(Model)[T.ë-C4 X],Price,Range,HP,Chargetime,China
Model_year,,,,,,,,,,,,,,,,,,,,,
I3_2013,-0.653139,0.108843,-0.000421,0.110618,0.077471,-0.084564,-0.222253,-0.440121,-0.658936,-0.694804,...,0.401657,-0.020141,0.379863,0.202845,-0.033595,-0.004918,0.018706,-0.00493,-0.011275,-0.26475
Leaf_2013,-0.338651,0.056435,-0.000218,0.057355,0.040169,-0.043846,-0.115238,-0.228202,-0.341657,-0.360254,...,0.208258,-0.010443,0.196958,0.105174,-0.017419,-0.00255,0.009699,-0.002556,-0.005846,-0.137272
Zoe_2013,-0.300877,0.05014,-0.000194,0.050957,0.035688,-0.038956,-0.102384,-0.202747,-0.303547,-0.32007,...,0.185028,-0.009278,0.174989,0.093443,-0.015476,-0.002266,0.008617,-0.002271,-0.005194,-0.121961
Fortwo_2013,-0.040054,0.006675,-0.000026,0.006784,0.004751,-0.005186,-0.01363,-0.026991,-0.04041,-0.04261,...,0.024632,-0.001235,0.023296,0.01244,-0.00206,-0.000302,0.001147,-0.000302,-0.000691,-0.016236
Model S_2013,-0.610865,0.101798,-0.000393,0.103458,0.072457,-0.079091,-0.207868,-0.411634,-0.616287,-0.649834,...,0.37566,-0.018838,0.355277,0.189716,-0.031421,-0.0046,0.017496,-0.00461,-0.010546,-0.247615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
up!_2023,-0.008052,0.001342,-0.000005,0.001364,0.000955,-0.001043,-0.00274,-0.005426,-0.008124,-0.008566,...,0.004952,-0.000248,0.004683,0.002501,-0.000414,-0.000061,0.000231,-0.000061,-0.000139,-0.003264
C40_2023,-0.034894,0.005815,-0.000022,0.00591,0.004139,-0.004518,-0.011874,-0.023514,-0.035204,-0.03712,...,0.021459,-0.001076,0.020294,0.010837,-0.001795,-0.000263,0.000999,-0.000263,-0.000602,-0.014144
EX30_2023,-0.057152,0.009524,-0.000037,0.009679,0.006779,-0.0074,-0.019448,-0.038512,-0.057659,-0.060798,...,0.035147,-0.001762,0.03324,0.01775,-0.00294,-0.00043,0.001637,-0.000431,-0.000987,-0.023167


In [86]:
marginal_effects[OLS.params.index].mean()

Intercept         -0.083051
C(Year)[T.2014]    0.013840
C(Year)[T.2015]   -0.000053
C(Year)[T.2016]    0.014066
C(Year)[T.2017]    0.009851
                     ...   
Price             -0.000625
Range              0.002379
HP                -0.000627
Chargetime        -0.001434
China             -0.033665
Length: 122, dtype: float64

In [87]:
elasticity = fun.elasticity(logit_data, OLS)
elasticity

KeyError: "['C(Year)[T.2014]', 'C(Year)[T.2015]', 'C(Year)[T.2016]', 'C(Year)[T.2017]', 'C(Year)[T.2018]', 'C(Year)[T.2019]', 'C(Year)[T.2020]', 'C(Year)[T.2021]', 'C(Year)[T.2022]', 'C(Year)[T.2023]', 'C(Model)[T.2008]', 'C(Model)[T.208]', 'C(Model)[T.3 Crossback]', 'C(Model)[T.3-dørs]', 'C(Model)[T.4]', 'C(Model)[T.5]', 'C(Model)[T.500]', 'C(Model)[T.Ariya]', 'C(Model)[T.Atto 3]', 'C(Model)[T.Avenger]', 'C(Model)[T.Berlingo]', 'C(Model)[T.Born]', 'C(Model)[T.C4]', 'C(Model)[T.C40]', 'C(Model)[T.Cabriolet]', 'C(Model)[T.Citigo]', 'C(Model)[T.Combo]', 'C(Model)[T.Corsa]', 'C(Model)[T.Dolphin]', 'C(Model)[T.E]', 'C(Model)[T.E-HS9]', 'C(Model)[T.EL6]', 'C(Model)[T.EQA]', 'C(Model)[T.EQB]', 'C(Model)[T.EQC]', 'C(Model)[T.EQE]', 'C(Model)[T.EQE SUV]', 'C(Model)[T.EQS]', 'C(Model)[T.EQS SUV]', 'C(Model)[T.EQV]', 'C(Model)[T.ES8]', 'C(Model)[T.ET5]', 'C(Model)[T.ET7]', 'C(Model)[T.EV6]', 'C(Model)[T.EV9]', 'C(Model)[T.EX30]', 'C(Model)[T.Enyaq iV]', 'C(Model)[T.Euniq]', 'C(Model)[T.Euniq5]',

In [ ]:
elasticity[OLS.params.index].mean()

Intercept    -7.693137
Price        -1.306372
Range          4.67946
HP           -0.359517
Chargetime   -1.547039
China        -0.162447
dtype: object

# Analysis on subsample

### The subsample consists of the 2023 market with:
The 5 highest market share models: Model Y, Model 3, Enyag iV, ID.4 and Q4 e-tron\
The 5 highest market share chinese models: 4, Euniq6, Atto 3, Marvel R and Dolphin\
The highest ccp model: Ocean\
The highest ccp chinese model: Seal\
The highest ccp korean model (so we have models outside EU, US and china): Ioniq 6\
The highest priced model: I7\
Polestar 2 as a random interest


In [ ]:
analysis_data = logit_data[logit_data['Year']==2023].copy()
analysis_data = analysis_data[analysis_data['Model'].isin(['I7', 'Model 3', 'Model Y', 'ID.4', 'Enyaq iV', 'Ocean' , '2', 'Ioniq 6', 'Q4 e-tron'
                                                           , '4', 'Euniq6', 'Atto 3', 'Marvel R', 'Dolphin', 'Seal'])]
analysis_data.reset_index(drop=True, inplace=True)
analysis_data

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,China,CCP,Model_year
0,5,2023,0.037958,Audi,Q4 e-tron,49.6,65.855307,28.1,28,SUV,C,DE,2349,1.0,0,0.011269,Q4 e-tron_2023
1,13,2023,0.000679,BMW,I7,60.1,186.519444,44.9,28,Sedan,F,DE,42,1.0,0,0.000987,I7_2023
2,17,2023,0.003587,BYD,Atto 3,42.0,31.000000,20.1,37,SUV,C,CN,222,1.0,1,0.002214,Atto 3_2023
3,18,2023,0.003280,BYD,Dolphin,42.7,23.323517,9.3,40,Hatchback,C,CN,203,1.0,1,0.003074,Dolphin_2023
4,20,2023,0.000517,BYD,Seal,54.0,32.062000,52.2,38,Sedan,D,CN,32,1.0,1,0.005576,Seal_2023
5,49,2023,0.004686,Fisker,Ocean,70.0,60.237920,56.3,35,SUV,D,DK,290,1.0,0,0.077537,Ocean_2023
6,69,2023,0.003199,Hyundai,Ioniq 6,56.7,43.629166,32.0,16,Sedan,D,KR,198,1.0,0,0.084790,Ioniq 6_2023
7,100,2023,0.003991,Maxus,Euniq6,35.4,37.950676,17.4,45,MPV,M,CN,247,1.0,1,0.000576,Euniq6_2023
8,117,2023,0.011990,MG,4,42.7,26.589191,24.1,41,Hatchback,C,CN,742,1.0,1,0.002143,4_2023
9,119,2023,0.003361,MG,Marvel R,38.8,30.235650,17.7,38,SUV,C,CN,208,1.0,1,0.001509,Marvel R_2023


In [ ]:
cross_elasticity = fun.cross_elasticity_1(analysis_data, OLS)
cross_elasticity[:50]

/Users/mikkeldollerup/Documents/GitHub/Bachelor/functions.py:195: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cross_elasticity_table.loc[(model_labels[i], model_labels[j], X.columns[k]), 'Cross_Elasticity'] = -coefficients[k] * X.iloc[j, k] * ccp.iloc[j]
/Users/mikkeldollerup/Documents/GitHub/Bachelor/functions.py:195: PerformanceWarning: indexing past lexsort depth may impact performance.
  cross_elasticity_table.loc[(model_labels[i], model_labels[j], X.columns[k]), 'Cross_Elasticity'] = -coefficients[k] * X.iloc[j, k] * ccp.iloc[j]


Cross_Elasticity
Model_year     Model_year                                
Q4 e-tron_2023 Q4 e-tron_2023 Intercept          0.089644
                              Price              0.021393
                              Range             -0.068866
                              HP                 0.004712
                              Chargetime         0.014742
                              China                   0.0
               I7_2023        Intercept          0.007854
                              Price              0.005309
                              Range             -0.007311
                              HP                  0.00066
                              Chargetime         0.001292
                              China                   0.0
               Atto 3_2023    Intercept          0.017616
                              Price              0.001979
                              Range             -0.011459
                              HP                 0.000662
                              Chargetime         0.003828
                              China              0.003087
               Dolphin_2023   Intercept          0.024457
                              Price              0.002067
                              Range             -0.016175
                              HP                 0.000425
                              Chargetime         0.005746
                              China              0.004286
               Seal_2023      Intercept           0.04436
                              Price              0.005154
                              Range             -0.037101
                              HP                 0.004332
                              Chargetime           0.0099
                              China              0.007773
               Ocean_2023     Intercept          0.616821
                              Price              0.134647
                              Range             -0.668741
                              HP                 0.064962
                              Chargetime         0.126793
                              China                   0.0
               Ioniq 6_2023   Intercept          0.674515
                              Price              0.106644
                              Range             -0.592346
                              HP                 0.040377
                              Chargetime         0.063384
                              China                   0.0
               Euniq6_2023    Intercept           0.00458
                              Price               0.00063
                              Range             -0.002511
                              HP                 0.000149
                              Chargetime          0.00121
                              China              0.000803
               4_2023         Intercept          0.017044
                              Price              0.001642

# Cost

In [ ]:
logit_data['Cost_OG']=fun.cost_original(logit_data, alpha)
logit_data['Cost']=fun.cost(logit_data, alpha)
logit_data

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,China,CCP,Model_year,Cost_OG,Cost
0,10,2013,0.002237,BMW,I3,29.5,25.00000,16.7,18,Hatchback,B,DE,1,1.0,0,0.144783,I3_2013,20.505806,-11.295887
1,132,2013,0.472036,Nissan,Leaf,32.8,25.36900,14.7,43,Hatchback,C,JP,211,1.0,0,0.135095,Leaf_2013,21.175532,0.000000
2,158,2013,0.205817,Renault,Zoe,36.5,17.31500,13.4,56,Hatchback,B,FR,92,1.0,0,0.200516,Zoe_2013,11.090824,0.000000
3,167,2013,0.002237,Smart,Fortwo,12.7,21.04575,8.0,60,Hatchback,A,DE,1,1.0,0,0.022899,Fortwo_2013,20.334942,0.000000
4,173,2013,0.250559,Tesla,Model S,60.9,71.95631,67.5,30,Liftback,F,US,112,1.0,0,0.408012,Model S_2013,59.291257,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329,187,2023,0.001277,Volkswagen,up!,25.6,17.48530,8.1,48,Hatchback,A,DE,79,1.0,0,0.004881,up!_2023,17.333791,0.000000
330,188,2023,0.008354,Volvo,C40,46.6,43.06649,40.2,28,SUV,C,SE,517,1.0,0,0.016984,C40_2023,42.539280,0.000000
331,189,2023,0.000065,Volvo,EX30,47.5,36.82450,26.8,28,SUV,B,SE,4,1.0,0,0.021896,EX30_2023,36.144829,0.000000
332,190,2023,0.031752,Volvo,XC40,45.7,43.92666,40.2,28,SUV,C,SE,1965,1.0,0,0.015324,XC40_2023,43.450994,0.000000


In [ ]:
logit_data[logit_data['Cost']>logit_data['Price']]

,ID,Year,Market_share,Manufacturer,Model,Range,Price,HP,Chargetime,Type,Segment,Country,Sales,Intercept,China,CCP,Model_year,Cost_OG,Cost
